In [11]:
import sys
import os
import time
import re
import json
from urllib.parse import quote as urlencode
from urllib.request import urlretrieve

import http.client as httplib 

from astropy.table import Table
import numpy as np

import pprint
pp = pprint.PrettyPrinter(indent=4)
import SciServer
from SciServer import CasJobs, SkyQuery, SciDrive, SkyServer
import requests
import pandas as pd
import astropy as ap
import astropy.io.fits as astrofits
import astropy.io.votable as astrovot
import astropy.wcs as astrowcs
import astropy.units as astrounits
import astropy.coordinates as astrocoords
import astropy.visualization as astrovis
import astropy.visualization.mpl_normalize as astromplnorm
import astropy.nddata as astronddata
import astropy.nddata.utils as astrondutils
import io
import pdb 
from IPython.core.display import display

import matplotlib.pyplot as mplplot

In [12]:
def mastQuery(request):

    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    #pdb.set_trace()
    requestString = urlencode(requestString)
    # opening the https connection
    conn = httplib.HTTPSConnection(server)
    
    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)
    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('UTF-8')

    # Close the https connection
    conn.close()
    return head,content

In [13]:
def HscCrossmatch(data):
    
    crossmatchInput = {"fields":[{"name":"ra","type":"float"},
                                 {"name":"dec","type":"float"}],
                       "data":data}
    
    request = {"service":"Mast.Hsc.Crossmatch",
               "data":crossmatchInput,
               'params':{
                   "raColumn":"ra",
                   "decColumn":"dec",
                   'radius':0.001,
                   'cache-breaker':10},
               "pagesize":1000,
               "page":1,
               "format":"json",
               "removecache":True}
    
    headers,outString = mastQuery(request)
    outData = json.loads(outString)
    
    return outData

In [14]:
def mastJson2Table_Filt(jsonObj):
    filt = 'A_F475W'
    Obj = [x for x in jsonObj['data'] if x.get(filt,None) != None]
    mastDataTable = Table()

    for col,atype in [(x['name'],x['type']) for x in jsonObj['fields']]:
        mastDataTable[col] = np.array([x.get(col,None) for x in Obj])
    
    return mastDataTable

In [15]:
def mastJson2Table(jsonObj):

    dataTable = Table()

    for col,atype in [(x['name'],x['type']) for x in jsonObj['fields']]:
        dataTable[col] = np.array([x.get(col,None) for x in jsonObj['data']])
        
    return dataTable

In [16]:
def getHSCMatches(matchId):

#     data = result['data']

#     # get the match id
#     matchId = data[0]['MatchID']
#     print(type(matchId))
    # get detailed results for chosen match id
    request = {'service':'Mast.HscMatches.Db',
               'params':{'input':matchId},
               'format':'json',
               'page':1,
               'pagesize':4}   

    headers,outString = mastQuery(request)

    outData = json.loads(outString)

    return outData

In [17]:
def HSCProducts(obsid):
    

#     obsid = Obs['obsid']

    productRequest = {'service':'Mast.Hsc.Db.Products',
                     'params':{'obsid':obsid},
                     'format':'json',
                     'pagesize':100,
                     'page':1}   

    headers,obsProductsString = mastQuery(productRequest)

    obsProducts = json.loads(obsProductsString)
    
    return obsProducts

In [18]:
query = "SELECT dr8objid, ra, dec FROM zoo2MainSpecz"
result = CasJobs.executeQuery(query, 'DR12', format='pandas')
ra = result.ra.values
dec = result.dec.values
#display(result)

In [21]:
#os.chdir('/home/idies/workspace/persistent/JSON')
bins = 4000
number = 0
ra_dec_list = [{"ra":ras,"dec":decs} for ras,decs in zip(ra,dec)]
data = ra_dec_list[number*bins:(number*bins)+bins]
info = HscCrossmatch(data)
astrotable = mastJson2Table_Filt(info)
#print(info)
print(astrotable)
# with open('Matching.txt', 'a+') as outfile:
#     json.dump(info, outfile)
#     outfile.write('\n')

   ra       dec    MatchID ... W3_F160W_Sigma W3_F160W_N _selected_
-------- --------- ------- ... -------------- ---------- ----------
192.0427 -1.747009  278667 ...           None       None       None


In [29]:
# os.chdir('/home/idies/workspace/persistent/JSON')
# with open('Matching.txt') as f:
#     lines = f.read().splitlines()
#     f.close()


# test = json.loads(lines[1])
# astrotable = mastJson2Table(test)
#print(astrotable.keys())
pdtable = astrotable.to_pandas()
matches = getHSCMatches(str(pdtable.loc[0,'MatchID']))
astrotable1 = mastJson2Table(matches)
print(astrotable1['ImageName'])

        ImageName         
--------------------------
hst_13324_06_acs_wfc_f475w
hst_13324_06_acs_wfc_f814w


In [32]:
print(astrotable.keys())
print(astrotable1.keys())

['ra', 'dec', 'MatchID', 'distanceArcMin', 'MatchRA', 'MatchDec', 'DSigma', 'AbsCorr', 'NumFilters', 'NumVisits', 'NumImages', 'StartMJD', 'StopMJD', 'TargetName', 'CI', 'CI_Sigma', 'KronRadius', 'KronRadius_Sigma', 'Extinction', 'W2_F300W', 'W2_F300W_Sigma', 'W2_F300W_N', 'W3_F350LP', 'W3_F350LP_Sigma', 'W3_F350LP_N', 'A_F435W', 'A_F435W_Sigma', 'A_F435W_N', 'W2_F450W', 'W2_F450W_Sigma', 'W2_F450W_N', 'A_F475W', 'A_F475W_Sigma', 'A_F475W_N', 'W3_F547M', 'W3_F547M_Sigma', 'W3_F547M_N', 'A_F550M', 'A_F550M_Sigma', 'A_F550M_N', 'A_F555W', 'A_F555W_Sigma', 'A_F555W_N', 'W3_F555W', 'W3_F555W_Sigma', 'W3_F555W_N', 'A_F606W', 'A_F606W_Sigma', 'A_F606W_N', 'W2_F606W', 'W2_F606W_Sigma', 'W2_F606W_N', 'A_F625W', 'A_F625W_Sigma', 'A_F625W_N', 'W2_F675W', 'W2_F675W_Sigma', 'W2_F675W_N', 'A_F775W', 'A_F775W_Sigma', 'A_F775W_N', 'A_F814W', 'A_F814W_Sigma', 'A_F814W_N', 'W2_F814W', 'W2_F814W_Sigma', 'W2_F814W_N', 'W3_F814W', 'W3_F814W_Sigma', 'W3_F814W_N', 'W2_F1042M', 'W2_F1042M_Sigma', 'W2_F1042M_

In [60]:
server='mast.stsci.edu'
conn = httplib.HTTPSConnection(server)

for row in first_astrotable:     

    # make file path
    outPath = '/home/idies/workspace/persistent'
    if not os.path.exists(outPath):
        os.makedirs(outPath)
    outPath += '/test'
        
    # Download the data
    uri = row['ImageID']
    conn.request("GET", "/api/v0/download/file?uri="+uri)
    resp = conn.getresponse()
    fileContent = resp.read()
    
    # save to file
    with open(outPath,'wb') as FLE:
        FLE.write(fileContent)
        
    # check for file 
    if not os.path.isfile(outPath):
        print("ERROR: " + outPath + " failed to download.")
    else:
        print("COMPLETE: ", outPath)


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U26') dtype('<U26') dtype('<U26')